In [3]:
%matplotlib notebook

from __future__ import division
from __future__ import print_function

# Required modules
import numpy as np
import matplotlib.pyplot as plt
import warnings

from hmmlearn.hmm import GaussianHMM, GMMHMM
from hmmlearn.utils import iter_from_X_lengths

from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import zero_one_loss
from sklearn.utils import shuffle

from scipy.io import loadmat

import scipy.io as sio   

matfn = 'trajectories_train.mat'
data = sio.loadmat(matfn)  


In [90]:
for i in data:
    print(i)


__header__
__version__
__globals__
key
xtrain
ytrain


ValueError: could not broadcast input array from shape (3,174) into shape (3)

In [57]:
len(data['xtrain'][0]) #1429 * 3 * 174
lengths = list(map(len, [i[0] for i in data['xtrain'][0]]))

# get the index of every different charactors.

def get_data(num):
    return data['xtrain'][data['ytrain'] == num]
# for i in range(0,26):
#     print(len(get_index(i)))


In [123]:
X_train = {}
X_len = {}
for i in range(1, 21):
    temp = np.vstack(get_data(i))
    X_train[i] = temp.reshape(-1, 3)
    X_len[i] = list(map(len, [j[0] for j in get_data(i)]))
    


In [124]:
print(X_train[1].shape)
a = X_train[1].reshape(-1,3)
a.shape

(14774, 3)


(14774, 3)

In [142]:
data['xtrain'][0][2].shape

(3, 152)

In [125]:
# X_train[1][12]
# print(len(X_train[1][12]))
# print(X_len[2])
# for i in X_len:
#     print(X_len[i])
    
a = set(len(i[0]) for i in data['xtrain'][0])
len(data['ytrain'][0])

1429

In [126]:
train_x = data['xtrain'][0]
train_y = data['ytrain'][0]

In [127]:
len(X_train[4])

11360

In [292]:
hmm = {}
for i in range(1, 21):
    hmm[i] = GaussianHMM(
                   init_params='mc',
                   n_iter=10000,
                   random_state=1)
 

In [293]:
X_train.keys()
X_train[1].shape, len(X_len[1])
X_train[1][0]

array([ 0.,  0.,  0.])

In [294]:
for i in X_train:
    print(len(X_len[i]))
    if i == 1:
        hmm[i].fit(X_train[i], lengths = X_len[i])
    else:
        hmm[i] = clone(hmm[i-1], safe=True).fit(X_train[i], lengths = X_len[i])

83
84
66
71
96
75
57
79
67
62
66
70
57
58
65
64
90
58
68
93


In [295]:
# copy from tutorial week7, COMP9418-17s2, W07 Tutorial
def log_likelihood(hmm, sequence):

    logprob_frame = hmm._compute_log_likelihood(sequence)
    logprob_sequence, _ =  hmm._do_forward_pass(logprob_frame)

    return logprob_sequence

In [296]:
res = 0
for j in range(1, 200):
    result = []
    for i in hmm:
        result.append(log_likelihood(hmm[i], data['xtrain'][0][j].T))
    if (result.index(max(result))+1 == data['ytrain'][0][j]):
        res += 1
print(res / 200)

0.4


In [189]:
print(data['xtrain'][0][1][0])
print(X_train[1][0])
a = 0
for i in X_train[1]:
    a += 1
    print(i)
    if(a == 178):
        a = 0
        print("---------------")
        break


[ 0.          0.          0.         -0.07826339 -0.12412581 -0.17610067
 -0.23146846 -0.29085021 -0.35649998 -0.42918926 -0.50656599 -0.58379663
 -0.65538145 -0.71662183 -0.76401185 -0.79503815 -0.80830325 -0.80398524
 -0.78379078 -0.75023264 -0.70614562 -0.65487873 -0.60034678 -0.54629232
 -0.49538117 -0.44906752 -0.40821721 -0.37366079 -0.34596534 -0.32453207
 -0.30690757 -0.28913901 -0.26707684 -0.23771859 -0.199886   -0.15432253
 -0.1034096  -0.0502495   0.00247697  0.05245081  0.09657678  0.1312428
  0.15484049  0.16997752  0.18337843  0.20451993  0.24441301  0.3142078
  0.42296063  0.57505293  0.76843153  0.99441076  1.23877049  1.48323203
  1.70697045  1.88893047  2.01151311  2.06489777  2.05078602  1.98429882
  1.89163597  1.80157977  1.73415633  1.69451523  1.67628358  1.66983272
  1.66813501  1.66782791  1.66779979  1.66780253  1.66780408  1.66780422
  1.66780199  1.66778     1.66761309  1.66662006  1.66198247  1.64495702
  1.59574971  1.48362055  1.28190522  0.99512393  0. 

In [206]:
pi0 = np.eye(1, 5)[0]
pi0

array([ 1.,  0.,  0.,  0.,  0.])

In [207]:
#init guess for EM
# transition matrix for latent variable

n_states = 20
pi0 = np.eye(1, 20)[0]
trans0 = np.diag(np.ones(n_states)) + np.diag(np.ones(n_states-1), 1)
trans0 /= trans0.sum(axis=1).reshape(-1, 1)

gmmhmm = GMMHMM(n_components=n_states, 
                n_mix=1,
                covariance_type='diag',
                init_params='mc',
                n_iter=500,
                random_state=rng)
gmmhmm.startprob_ = pi0
gmmhmm.transmat_  = trans0

In [268]:
### SOLUTION ###
class GenerativeClassifierHMM(BaseEstimator, ClassifierMixin):

    def __init__(self, hmm=GaussianHMM()):

        self.hmm = hmm
        self.class_cond_hmms_ = []

    def fit(self, sequences, lengths):

        # assume labels are values are already
        # standardized between 0 and n_classes-1

        self.logprior = np.log(1 / 20)

        for c in range(1, 21):
            print(c)
            class_cond_hmm = clone(self.hmm, safe=True)
            class_cond_hmm.fit(sequences[c], lengths=lengths[c])

            self.class_cond_hmms_.append(class_cond_hmm)

        return self

    def predict(self, sequences):

        log_likelihood_ = log_likelihoods_cond(self.class_cond_hmms_, 
                                               sequences)

        log_post_unnorm = log_likelihood_ + self.logprior.reshape(-1, 1)

        return np.argmax(log_post_unnorm, axis=0)

In [269]:
import warnings
warnings.filterwarnings('ignore')

hmm_classifier = GenerativeClassifierHMM(gmmhmm)
hmm_classifier.fit(X_train, 
                   X_len)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


GenerativeClassifierHMM(hmm=GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=None,
    covars_weight=None, init_params='mc', means_prior=0.0,
    means_weight=0.0, min_covar=0.001, n_components=20, n_iter=500,
    n_mix=1, params='stmcw',
    random_state=<mtrand.RandomState object at 0x0000016F6B247870>,
    startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False,
    weights_prior=1.0))

In [271]:
# def log_likelihoods(hmm, sequences):
#     ll = lambda seq: log_likelihood(hmm, seq)
#     return np.fromiter(map(ll, sequences), dtype='float64')

# def log_likelihoods_cond(cond_hmms, sequences):
#     ll = lambda hmm: log_likelihoods(hmm, sequences)
#     return np.vstack(map(ll, cond_hmms))

res = 0
for j in range(1, 200):
    result = []
    for i in hmm_classifier.class_cond_hmms_:
        result.append(log_likelihood(i, data['xtrain'][0][j].T))
    if (result.index(max(result))+1 == data['ytrain'][0][j]):
        res += 1
    print(result)
print(res / 200)   


[nan, nan, nan, -3291.0575648776235, nan, nan, nan, nan, nan, nan, nan, -7462.0406409778552, nan, -9679.9173028804253, nan, -8296.4851001209154, nan, nan, -3440.1835051444646, -6764.6954246078994]
[nan, nan, nan, -257.36240715683101, nan, nan, nan, nan, nan, nan, nan, -3569.8759625317125, nan, -5531.1038762250719, nan, -4237.2202745868744, nan, nan, 530.54288491616046, -5644.6876625760133]
[nan, nan, nan, -1659.9195225214805, nan, nan, nan, nan, nan, nan, nan, -5218.1511270341271, nan, -7483.2235721307698, nan, -4759.241919150616, nan, nan, -1319.2431046405272, -4731.8762502407935]
[nan, nan, nan, -1337.9824054977537, nan, nan, nan, nan, nan, nan, nan, -5014.6371705215033, nan, -7235.1806560438226, nan, -4345.0661951100619, nan, nan, -342.48703751586402, -6519.7301113453468]
[nan, nan, nan, -1099.2841748343678, nan, nan, nan, nan, nan, nan, nan, -4654.1206499467626, nan, -5559.8324207823944, nan, -3268.345944992388, nan, nan, -624.61402688872806, -5517.5664733871436]
[nan, nan, nan, -1

[nan, nan, nan, -1094.5351791826035, nan, nan, nan, nan, nan, nan, nan, -7071.7445698005413, nan, -6421.6959050563974, nan, -6291.6138112047684, nan, nan, 201.13143960224681, -8384.8036218748603]
[nan, nan, nan, -2009.720702207984, nan, nan, nan, nan, nan, nan, nan, -7249.7575040156999, nan, -6608.211747213426, nan, -3902.5307302465321, nan, nan, -615.94090922732778, -6225.7064576749463]
[nan, nan, nan, -1290.513197505494, nan, nan, nan, nan, nan, nan, nan, -4886.5437692714067, nan, -6848.8310908621124, nan, -2860.0353389063098, nan, nan, -1050.840309524403, -4361.4339710312579]
[nan, nan, nan, -443.59503827327666, nan, nan, nan, nan, nan, nan, nan, -4837.079815365686, nan, -6172.213094934913, nan, -3679.2229504232037, nan, nan, 305.49305795914108, -4294.3558970022505]
[nan, nan, nan, -2031.003262927786, nan, nan, nan, nan, nan, nan, nan, -6468.1455265029053, nan, -8047.5796769050567, nan, -5195.8054913245451, nan, nan, -908.97453316648182, -5584.2606239551214]
[nan, nan, nan, 40.12211

[nan, nan, nan, -1325.3524692182725, nan, nan, nan, nan, nan, nan, nan, -6022.5136570670857, nan, -6940.7708027147373, nan, -4145.2087692941031, nan, nan, -250.40364658894271, -5722.7785375052599]
[nan, nan, nan, 611.63826222671696, nan, nan, nan, nan, nan, nan, nan, -3139.2159499069503, nan, -3964.313273625442, nan, -2630.2535535501092, nan, nan, 1190.9254493254948, -4815.5362517372923]
[nan, nan, nan, -1323.8804912233084, nan, nan, nan, nan, nan, nan, nan, -7219.2561618328937, nan, -7535.4996576496715, nan, -4323.8376489961101, nan, nan, -676.76832644943408, -5542.5315400643076]
[nan, nan, nan, -1289.5088710608031, nan, nan, nan, nan, nan, nan, nan, -7644.7181228219397, nan, -7899.2076622610048, nan, -4681.0983499479489, nan, nan, -640.0438885203481, -6616.5299716460031]
[nan, nan, nan, -1552.6534638487367, nan, nan, nan, nan, nan, nan, nan, -8684.216855957422, nan, -7504.5441315322423, nan, -5119.9838603855351, nan, nan, 154.18815261875594, -7755.3481561348544]
[nan, nan, nan, -2162

[nan, nan, nan, -775.96152183727224, nan, nan, nan, nan, nan, nan, nan, -6750.1797808496285, nan, -6370.5822936765362, nan, -3415.5876970515819, nan, nan, 506.56750338309223, -6845.8898350787476]
[nan, nan, nan, 95.648245159367065, nan, nan, nan, nan, nan, nan, nan, -3217.380548003975, nan, -4192.1637963370786, nan, -4281.236826708835, nan, nan, 1607.6749196695332, -4606.9544986219798]
[nan, nan, nan, -2311.1653782470057, nan, nan, nan, nan, nan, nan, nan, -8666.816598958676, nan, -6046.8773539155782, nan, -5638.0418845461281, nan, nan, -1585.3477950660786, -8810.3425153498338]
[nan, nan, nan, -1026.5165798588025, nan, nan, nan, nan, nan, nan, nan, -5512.9678139388707, nan, -7057.95389747218, nan, -3937.8852269006507, nan, nan, -825.42130567595916, -5524.0984618214306]
[nan, nan, nan, -1637.2387596155229, nan, nan, nan, nan, nan, nan, nan, -7717.0304272425865, nan, -10048.1218631351, nan, -6686.488309944506, nan, nan, -1241.4472722602018, -6077.384089068124]
[nan, nan, nan, -2131.91036

[nan, nan, nan, -3100.0539808966637, nan, nan, nan, nan, nan, nan, nan, -9291.5155483060225, nan, -12954.040896019607, nan, -10905.503163289542, nan, nan, -1923.1615837644863, -8525.7003993843573]
[nan, nan, nan, -1490.8396571814662, nan, nan, nan, nan, nan, nan, nan, -6021.0514451238159, nan, -7044.456962351529, nan, -3688.6614182398698, nan, nan, -852.19455523974841, -5641.8594908404521]
[nan, nan, nan, -4013.0612128800317, nan, nan, nan, nan, nan, nan, nan, -9079.8372635674714, nan, -12834.297766014659, nan, -11772.52290721417, nan, nan, -3330.9186278127358, -9247.7804903272336]
[nan, nan, nan, -1559.7093662780942, nan, nan, nan, nan, nan, nan, nan, -5968.4539120474901, nan, -7849.5974661030632, nan, -4685.3324383005202, nan, nan, -247.76962707454561, -5983.5441340041189]
[nan, nan, nan, -1457.1312898309254, nan, nan, nan, nan, nan, nan, nan, -7731.4636306757075, nan, -7881.8284904129723, nan, -4795.4176345957767, nan, nan, -299.41148488849842, -6918.3899001179716]
[nan, nan, nan, -

In [228]:
a = label_enc.transform(data['ytrain'].T)

C:\Users\Yu\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [242]:
 data['xtrain'][0][label_enc.transform(data['ytrain'].T) == 1]

C:\Users\Yu\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   8.95969503e-02,   1.14810117e-01,
          1.29470002e-01,   1.30514891e-01,   1.17025300e-01,
          8.72788977e-02,   3.76322484e-02,  -3.55600499e-02,
         -1.32885185e-01,  -2.50571168e-01,  -3.82063217e-01,
         -5.20443714e-01,  -6.58851072e-01,  -7.89101354e-01,
         -9.01327089e-01,  -9.86099257e-01,  -1.03772673e+00,
         -1.05659962e+00,  -1.04941224e+00,  -1.02734343e+00,
         -1.00294007e+00,  -9.86536885e-01,  -9.83233192e-01,
         -9.91870311e-01,  -1.00688641e+00,  -1.02184159e+00,
         -1.03176402e+00,  -1.03249719e+00,  -1.01840545e+00,
         -9.82012680e-01,  -9.17517034e-01,  -8.25549945e-01,
         -7.14199629e-01,  -5.94699350e-01,  -4.75682403e-01,
         -3.61062511e-01,  -2.52359279e-01,  -1.51783288e-01,
